API Documentation: https://rapidapi.com/fkal094tiokg09w3vi095i/api/Airbnb%20Scraper%20API

In [2]:
## Before, make sure that the GCP libraries are installed

## pip install google
## pip install google.cloud
## pip install google.cloud.bigquery
## pip install google.cloud.storage
## pip install datetime, timedelta
## pip install pandas
## pip install db-dtypes

In [23]:
## Importing libraries to execute querying on BigQuery

import os
from google.cloud import bigquery
from google.oauth2 import service_account
from credentials import bq_path

In [ ]:
## Importing Credentials from Google Cloud

CREDENTIALS = service_account.Credentials.from_service_account_file(bq_path)
BIGQUERY = bigquery.Client(credentials=CREDENTIALS)

This section aims to create automated variables to perform the scrapping more sustainably

In [5]:
## Creating check-in and check-out variables

from datetime import datetime, timedelta
import pandas as pd

currentDate =  datetime.today()

## Creating list to save CheckIn and Checkout combinations

checkInAndOutDates = [
        {'checkin': (currentDate + timedelta(days=30)).strftime('%Y-%m-%d'), 
         'checkout': (currentDate + timedelta(days=60)).strftime('%Y-%m-%d')
        },
        #{'checkin': (currentDate + timedelta(days=60)).strftime('%Y-%m-%d'), 
         #'checkout': (currentDate + timedelta(days=90)).strftime('%Y-%m-%d')
        #},
        ##{'checkin': (currentDate + timedelta(days=90)).strftime('%Y-%m-%d'), 
        ## 'checkout': (currentDate + timedelta(days=120)).strftime('%Y-%m-%d')
        ##}
    ]


In [6]:
## Creating location variables 
## Query collecting Tabas Operating Neighbourhoods

sql =  """
        SELECT 
            DISTINCT CONCAT(city, ', ', neighborhood) AS city_and_neighbourhood_search
        FROM `tabas-dw.master_data.dim_tabas_buildings_and_apartments`
        LIMIT 1
        """

In [ ]:
## Creating dataframe neighbourhoods to write the results

neighbourhoods = BIGQUERY.query(sql).result().to_dataframe()
neighbourhoods = neighbourhoods.values.tolist()

/home/tabas/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


This sections aims to make a GET Request to the Airbnb Scraper API

In [8]:
## Creating dataframe df to write the following loop results

df = pd.DataFrame(columns=['badges'
                           , 'coordinates'
                            , 'id'
                            , 'images'
                            , 'price'
                            , 'rating'
                            , 'reviews'
                            , 'roomTitle'
                            , 'roomType'
                            , 'subTitle'
                            , 'title'
                            , 'url'
                            , 'location'
                            , 'checkin'
                            , 'checkout'
                            , 'adults'
                            , 'scrappedPage'
                            , 'extractionTimestamp'
                        ])

In [25]:
import requests
import pandas as pd
import json

## Define the variables to access Airbnb Scraper API

url = "https://airbnb-scraper-api.p.rapidapi.com/airbnb_search_stays_v2"

headers = {
    'x-rapidapi-key': "3518f08619msh6dcfedfcbed6577p1d7ff2jsn6219c0aab48e",
    'x-rapidapi-host': "airbnb-scraper-api.p.rapidapi.com"
}


for i in range(len(neighbourhoods)):

    for j in range(len(checkInAndOutDates)):

        ## The cursor is an unique indicator of the page, this helps the API to know which page to scrap next
        ## It is re-set no None on a new request

        cursor = None
        hasNextPage = True

        ## Creating the following dataframe to follow-up the amount of pages scrapped
        ## It is re-set to empty on a new request

        cursorDataFrame = []

        ## The following parameter estipulates the limit amount of pages to be scrapped
        ## , if desired

        pageLimitation = 2

        while hasNextPage and len(cursorDataFrame) < pageLimitation:
    
            querystring = {
                "location": neighbourhoods[i][0],                  # Desired location
                "checkIn": checkInAndOutDates[j]['checkin'],       # Check-in Date
                "checkOut": checkInAndOutDates[j]['checkout'],      # Check-out Date
                "adults": "2",                                      # Number of adults
                "roomType": "2",                                    # Type of Acommodation: Entire Space
                "cursor": cursor
            }

            ## Logging the location being sent to the request

            print("Getting Request: ", querystring)

            ## Send GET request to the API

            response = requests.get(url, headers=headers, params=querystring)

            ## Extract the JSON text data into the variable 'data'

            data = response.text

            ## Convert JSON into a Pandas Dataframe

            data = json.loads(data)
            extracted = pd.DataFrame.from_dict(data['data'])

            ## Setting the new cursor value to scrape the following page

            cursor = data['pageInfo']['endCursor']
            hasNextPage = data['pageInfo']['hasNextPage']

            ## Add the cursor result to the cursor dataframe

            cursorDataFrame.append(cursor)

            ## Create new columns on extracted DataFrame to append API variables

            extracted['location'] = neighbourhoods[i][0]
            extracted['checkin'] = checkInAndOutDates[j]['checkin']
            extracted['checkout'] = checkInAndOutDates[j]['checkout']
            extracted['adults'] = 2
            #extracted['roomType'] = 2     -> Information already exists on JSON
            extracted['scrappedPage'] = len(cursorDataFrame) + 1 
            extracted['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')

            ## Add the result to the previous created Dataframe
            
            df = df = pd.concat([df, extracted])

            print("Successfully added API request to DataFrame")

        else:
            if len(cursorDataFrame) > 0:
                print("Sucessfully scraped ", len(cursorDataFrame), " pages")
            else: 
                print("Error on API request")

print("End of API request")


Getting Request:  {'location': 'São Paulo, Jardins', 'checkIn': '2025-02-22', 'checkOut': '2025-03-24', 'adults': '2', 'roomType': '2', 'cursor': None}
Successfully added API request to DataFrame
Getting Request:  {'location': 'São Paulo, Jardins', 'checkIn': '2025-02-22', 'checkOut': '2025-03-24', 'adults': '2', 'roomType': '2', 'cursor': 'eyJzZWN0aW9uX29mZnNldCI6MCwiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0='}
Successfully added API request to DataFrame
Sucessfully scraped  2  pages
End of API request


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   badges               17 non-null     object
 1   coordinates          18 non-null     object
 2   id                   18 non-null     object
 3   images               18 non-null     object
 4   price                18 non-null     object
 5   rating               18 non-null     object
 6   reviews              18 non-null     object
 7   roomTitle            18 non-null     object
 8   roomType             18 non-null     object
 9   subTitle             18 non-null     object
 10  title                18 non-null     object
 11  url                  18 non-null     object
 12  location             18 non-null     object
 13  checkin              18 non-null     object
 14  checkout             18 non-null     object
 15  adults               18 non-null     object
 16  scrappedPa

This section focus on the Ingestion of the resulting DataFrame to a specific Datalake.

In [44]:
## Importing Credentials from Google Cloud

from google.cloud import storage
STORAGE = storage.Client(credentials=CREDENTIALS)

In [ ]:
# Acessing Bucket Path

bucket = STORAGE.get_bucket('us-central1-airflow-v5-30f2abcd-bucket')

Próximos Passos:
- Esconder credenciais ou caminho das credenciais
- Tratar dataframe
- Entender possibilidades de Ingestão deste dataframe
- Transformar dataframe em delta table?
- Acrescentar biblioteca de log

Cálculo de viabilidade:
Cenário 1:
- Tabas opera em 35 bairros distintos;
- 3 requisições distintas por bairro (supondo 3 datas de check-in/check-out)
- 6 páginas em média por bairro/check-in/ceck-out
- executando 1 vez por semana
TOTAL DE REQUISIÇÕES: 2520 

Cenário 2:
- Tabas opera em 35 bairros distintos;
- 4 requisições distintas por bairro (supondo 4 datas de check-in/check-out)
- 10 páginas em média por bairro/check-in/ceck-out
- executando 1 vez por semana
TOTAL DE REQUISIÇÕES: 5600

Cenário 3:
- Tabas opera em 35 bairros5 distintos;
- 5 requisições distintas por bairro (supondo 5 datas de check-in/check-out)
- 8 páginas em média por bairro/check-in/ceck-out
- executando 6 vezes por mês
TOTAL DE REQUISIÇÕES: 8400

